# Working Notebook 1

# Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns


import matplotlib.pyplot as plt
import scipy.stats as stats

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import QuantileTransformer, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,LassoLars, TweedieRegressor

import env

# Acquire data

In [ ]:
def get_zillow_data():
        ''' Acquire Zillow data using properties_2017 table from Code up Data Base. Columns bedroomcnt, 
            bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips 
        '''
   
         # sql query for acquisition
        sql_query = """
        SELECT calculatedfinishedsquarefeet,bathroomcnt,bedroomcnt,taxvaluedollarcnt

        FROM properties_2017
        LEFT JOIN propertylandusetype USING(propertylandusetypeid)
        LEFT JOIN predictions_2017 USING(parcelid)
        WHERE (propertylandusetype.propertylandusedesc LIKE ("%%Single%%")) 
            AND (predictions_2017.transactiondate like "2017%%");
        """
        # Acquisition
        df = pd.read_sql(sql_query, env.get_connection('zillow'))
        return df

In [ ]:
# had to adress % by using %% so it could be read by notebook

In [ ]:
# import data
df = get_zillow_data()

In [ ]:
# 52441 observations
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
# info
df.info()

<div class="alert alert-info"> There seems to be nulls in calculatedfiniched square feet

In [ ]:
df.isnull().sum()

<div class="alert alert-info"> There are 82 nulls in calculatedfinishedsquarefeet and one in taxvalluedollarcnt I could drop them or try to fill in with mean or median value.

In [ ]:
df.describe().T

<div class="alert alert-info"> There seems to be bathrooms/ bedrooms with min of 0 and max above what ight be consider a single family dweling

In [ ]:
# rename columns
df = df.rename(columns={'bedroomcnt': 'bedrooms','bathroomcnt': 'bathrooms',
            'calculatedfinishedsquarefeet': 'squarefeet','taxvaluedollarcnt': 'tax_value'})

In [ ]:
df = df.dropna()

In [ ]:
df= df[~(df.bathrooms==0) & ~(df.bedrooms ==0)]

In [ ]:
def handle_outliers(df):
    """Manually handle outliers that do not represent properties likely for 99% of buyers and zillow visitors"""
    df = df[df.bathrooms <= 5.5]
    
    df = df[df.bedrooms<= 6]
    
    return df


In [ ]:
df = handle_outliers(df)
df

# split data for exploration

In [ ]:
def split_data(df):
    '''
    split_data takes in data Frame and splits into  train , validate, test.
    The split is 20% test 80% train/validate. Then 30% of 80% validate and 70% of 80% train.
    Aproximately (train 56%, validate 24%, test 20%)
    Returns train, validate, and test 
    '''
    # split test data from train/validate
    train_and_validate, test = train_test_split(df, random_state=123, test_size=.2)

    # split train from validate
    train, validate = train_test_split(train_and_validate, random_state=123, test_size=.3)
                                   
    return train, validate, test

In [ ]:
train, validate, test = split_data(df)

In [ ]:
train.info()

In [ ]:
train.describe().T

In [ ]:
train.head()

# Explore

In [ ]:
sns.pairplot(train)

In [ ]:
# correaltion between variables
zillow_corr = train.corr(method='spearman')
zillow_corr

In [ ]:
# pass my correlation matrix to Seaborn's heatmap 
kwargs = {'alpha':.9,
          'linewidth':3, 
          'linestyle':'-',
          'linecolor':'black'}
sns.heatmap(zillow_corr, cmap='Purples', annot=True, 
            mask=np.triu(zillow_corr), **kwargs)

In [ ]:
def plot_variable_pair(df):
    columns = ['bedrooms',
 'bathrooms',
 'squarefeet']
    for i, col in enumerate(columns):
        sns.lmplot(data=df, x=col, y='tax_value', line_kws={'color':'red'})
        plt.show()

In [ ]:
plot_variable_pair(train)

In [ ]:
def plot_categorical_and_continuous_vars(df, cat_vars, cont_vars):
    for col in cat_vars:
        for col2 in cont_vars:
            fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(16,6))
            fig.suptitle(f'{col} vs. {col2}')
            sns.boxplot(data=df, x=col, y=col2, ax=ax1)
            sns.violinplot(data=df, x=col, y=col2, ax=ax2)
            sns.barplot(data=df, x=col, y=col2, ax=ax3)
            plt.show()

In [ ]:
# set categories
cat_vars =['bathrooms','bedrooms']
cont_vars =['tax_value','squarefeet']

In [ ]:
plot_categorical_and_continuous_vars(train, cat_vars, cont_vars)

In [ ]:
def show_cat_vs_cont(df, cat_vars=cat_vars, cont_vars=cont_vars):
    print('Categorical vs Continuous Variables:')
    #number = 1
    palettes = ['flare', 'Blues_r', 'PuRd_r', 'Accent']
    for j, cont in enumerate(cont_vars):
        plt.figure(figsize=(20,4))
        plt.suptitle(cont)
        for i, cat in enumerate(cat_vars):
            plt.subplot(1, 4, i+1)
            sns.barplot(data=df, x=cat, y=cont, palette=palettes[j])
            plt.title(cat + ' vs ' + cont)
        plt.show()

In [ ]:
show_cat_vs_cont(train, cat_vars,cont_vars)

In [ ]:
plt.figure(figsize=(8, 12))
heatmap = sns.heatmap(train.corr(method='spearman')[['tax_value']].sort_values(by='tax_value', ascending=False), vmin=-1, vmax=1, annot=True, cmap='Purples')
heatmap.set_title('Features Correlating with Home Price', fontdict={'fontsize':18}, pad=16);

In [ ]:
sns.countplot(x='bathrooms', data=train)

In [ ]:
sns.countplot(x='bedrooms', data=train)

# Stats

In [ ]:
above_mean_bathrooms = train.bathrooms[train.bathrooms > train.bathrooms.mean()]
overall_mean = train.bathrooms.mean()

In [ ]:
alpha=0.05
t, p = stats.ttest_1samp(above_mean_bathrooms, overall_mean)

print(t, p/2)

In [ ]:
# pearsonr r for continuous variables
for i in cont_vars:
    α =0.05
    corr, p = stats.pearsonr(train[i], train.tax_value)
    print('_____________________________________________________')
    print('HYPOTHESIS')
    print(f'H0: There is no significant difference between tax_value in different {i}')
    print(f'Ha:(There is a significant difference between tax_value in different {i}')
    print(f'{i} correlation {corr}, p-val{p}')
    if p < α:
        print('We reject the null hypothesis.')
        print(f'There is a correlation between tax_value and {i}')
    else:
        print('We fail to reject the null hypothesis.')
        print(f'There is no correlation between tax_value and {i}')

In [ ]:
for i in cat_vars:
    α =0.05
    corr, p = stats.pearsonr(train[i], train.tax_value)
    print('_____________________________________________________')
    print('HYPOTHESIS')
    print(f'H0: There is no significant difference between tax_value in different {i}')
    print(f'Ha:(There is a significant difference between tax_value in different {i}')
    print(f'{i} correlation {corr}, p-val{p}')
    if p < α:
        print('We reject the null hypothesis.')
        print(f'There is a correlation between tax_value and {i}')
    else:
        print('We fail to reject the null hypothesis.')
        print(f'There is no correlation between tax_value and {i}')

# Model

### Scale Data

In [ ]:
def scale_data(train, 
               validate, 
               test, 
               columns_to_scale=['bedrooms', 'bathrooms', 'squarefeet']):
    '''
    scale_data takes in train , validate, test data  and returns their scaled counterparts.
    '''
    # create copies of our original data
    train_scaled = train.copy()
    validate_scaled = validate.copy()
    test_scaled = test.copy()
    #create the scaler
    scaler = QuantileTransformer(output_distribution='normal')
    # fit the scaler into train data
    scaler.fit(train[columns_to_scale])
    
    # applying the scaler to train, validate, and test data
    train_scaled[columns_to_scale] = pd.DataFrame(scaler.transform(train[columns_to_scale]),
                                                  columns=train[columns_to_scale].columns.values).set_index([train.index.values])
                                                  
    validate_scaled[columns_to_scale] = pd.DataFrame(scaler.transform(validate[columns_to_scale]),
                                                  columns=validate[columns_to_scale].columns.values).set_index([validate.index.values])
    
    test_scaled[columns_to_scale] = pd.DataFrame(scaler.transform(test[columns_to_scale]),
                                                 columns=test[columns_to_scale].columns.values).set_index([test.index.values])
    
    return train_scaled, validate_scaled, test_scaled

In [ ]:
X_train_scaled, X_validate_scaled, X_test_scaled = scale_data(train,validate,test)

In [ ]:

# Setup X and y
X_train_scaled = X_train_scaled.drop(columns='tax_value')
y_train = train.tax_value

X_validate_scaled = X_validate_scaled.drop(columns='tax_value')
y_validate = validate.tax_value

X_test_scaled = X_test_scaled.drop(columns='tax_value')
y_test = test.tax_value

In [ ]:
# Turn everything into a dataframe
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train_scaled.columns)
X_validate_scaled = pd.DataFrame(X_validate_scaled, columns=X_validate_scaled.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test_scaled.columns)

### Select Features for model

In [ ]:
def select_Kbest(X, y, k=2):   
    '''
    select_Kbest takes in :
    X: a dataframe representing numerical independent features
    y: a pandas Series representing a target variable
    k: a keyword argument defaulted to 2 for the number of features
    
    returns: a list of the selected features using SelectBest 
    '''
    # model
    kbest = SelectKBest(f_regression, k=k)
    # fit
    kbest.fit(X, y)
    # mask of selection
    feature_mask = kbest.get_support()
    rank = X.columns[feature_mask]
    
    # convert to list
    rank = rank.to_list()


    return f'TOP {k} features: {rank}'

In [ ]:
select_Kbest(X_train_scaled,y_train, k=2)

### Simple model

In [ ]:
predictions = pd.DataFrame({
    'actual': validate.tax_value
}) 

In [ ]:
# X must be 2-d array
#X_train_scaled = X_train_scaled[['bathrooms']]
# y can be 1-d array
#y_train = train.tax_value

# 1. make the thing
lm = LinearRegression()
# 2. fit the thing
lm.fit(X_train_scaled[['bathrooms']], y_train)
# 3. use the thing (make predictions)
#X_validate = validate[['taxamount']]
predictions['simple_lm'] = lm.predict(X_validate_scaled[['bathrooms']])

In [ ]:
# look into modesl make up 
lm.coef_, lm.intercept_

In [ ]:
print(f'tax_value = {lm.coef_}*bathrooms + {lm.intercept_}')

In [ ]:
predictions

### Multiple + RFE

In [ ]:
def rfe(X, y, k=2):
    ''' 
    rfe takes in:
    X: a dataframe representing numerical independent features
    y: a pandas Series representing a target variable
    k: a keyword argument defaulted to 2 for the number of features
    
    returns: a list of the selected features using RFE
    '''
    # Model
    model = LinearRegression()
    rfe = RFE(model, n_features_to_select= k)
    #fit
    rfe.fit(X, y)
    # mask selection
    mask = rfe.get_support()
    
    return X.columns[mask]

In [ ]:
rfe(X_train_scaled, y_train, k=2)

In [ ]:


lm = LinearRegression()
k = 2

### 1. Transform our X
rfe = RFE(lm, n_features_to_select=2)
rfe.fit(X_train_scaled, y_train)
print('selected top 2 features:', X_train_scaled.columns[rfe.support_])
X_train_rfe = rfe.transform(X_train_scaled)

In [ ]:
# 1. Transform our X
X_train_rfe = rfe.transform(X_train_scaled)
# 2. Use the transformed x in our model
lm.fit(X_train_rfe, y_train)

# 3. Make predictions
X_validate_rfe = rfe.transform(X_validate_scaled)
predictions['multiple_rfe'] = lm.predict(X_validate_rfe)

predictions.head()

In [ ]:
# look into models make up
lm.coef_, lm.intercept_

In [ ]:
print(f'tax_value = {lm.coef_[0]:.2f} x squarefeet + {lm.coef_[1]:.2f} x bedroom + {lm.intercept_:.2f}')

### Poly Degree2

In [ ]:
X_train_scaled

In [ ]:
# 1. Generate Polynomial Features
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
poly.fit(X_train_scaled)
X_train_poly = pd.DataFrame(
    poly.transform(X_train_scaled),
    columns=poly.get_feature_names(X_train_scaled.columns),
    index=train.index,
)
X_train_poly.head()

In [ ]:
# 2. Use the features
lm = LinearRegression()
lm.fit(X_train_poly, y_train)

X_validate_poly = poly.transform(X_validate_scaled)
predictions['polynomial degree 2'] = lm.predict(X_validate_poly)
predictions

# Poly interactions_only 

In [ ]:
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
poly.fit(X_train_scaled)
X_train_poly = pd.DataFrame(
    poly.transform(X_train_scaled),
    columns=poly.get_feature_names(X_train_scaled.columns),
    index=train.index,
)
lm = LinearRegression()
lm.fit(X_train_poly, y_train)

X_validate_poly = poly.transform(X_validate_scaled)
predictions['polynomial only interaction'] = lm.predict(X_validate_poly)

pd.Series(lm.coef_, index=poly.get_feature_names(X_train_scaled.columns)).sort_values()

In [ ]:
predictions

## Laso Lars

In [ ]:
# create the model object
lars = LassoLars(alpha=1)

# fit the model to our training data
lars.fit(X_train_scaled, y_train)

# predict validate
X_validate_pred_lars = lars.predict(X_validate_scaled)

# Add lassolars predictions to our predictions DataFrame
predictions['lasso_lars'] = X_validate_pred_lars

In [ ]:
predictions

# GLM

In [ ]:
# create the model object
glm = TweedieRegressor(power=1, alpha=0)

# fit the model to our training data
glm.fit(X_train_scaled, y_train)

# predict validate
X_validate_predict_glm = glm.predict(X_validate_scaled)

# Add lassolars predictions to our predictions DataFrame
predictions['glm'] = X_validate_predict_glm

### Baseline

In [ ]:
# add a baseline model
predictions['baseline'] = train.tax_value.mean()

In [ ]:
predictions

# Evaluate

In [ ]:
def calculate_mse(y_predicted):
    return mean_squared_error(predictions.actual, y_predicted)

predictions.apply(calculate_mse).sort_values()

In [ ]:
pd.options.display.float_format = '{:,.3f}'.format
def calculate_RMSE(y_predicted):
    return mean_squared_error(predictions.acual,y_predicted)
predictions.apply(calculate_mse).sort_values()**.5

First itteration of models Polynomial degree2 was best, Then glm, then lasso_lars then baseline

polynomial degree 2           449,805.551

glm                           452,358.958

polynomial only interaction   459,250.729